In [32]:
import math
import random
import time

class Player():
  def __init__(self,letter):
    self.letter=letter
  def get_Move(self,game):
    pass

class HumanPlayer(Player):
  def __init__(self,letter):
    super().__init__(letter)

  def get_move(self,game):
    valid_Square=Falseval=None
    while not valid_Square:
      square=input(self.letter+"\'s turn. Input move (0-8): ")
      try:
        val=int(square)
        if val not in game.available_moves():
          raise ValueError
        valid_Square=True
      except ValueError:
        print('Invalid square, Try again.')
      return val


class AI(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())
        else:
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        max_player = self.letter 
        other_player = 'O' if player == 'X' else 'X'

        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (
                        state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -math.inf}  
        else:
            best = {'position': None, 'score': math.inf}  
        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)  

            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move 

            if player == max_player: 
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best



class TicTacToe():
    def __init__(self):
        self.board = self.make_board()
        self.current_winner = None

    @staticmethod
    def make_board():
        return [' ' for _ in range(9)]

    def print_board(self):
        for row in [self.board[i*3:(i+1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # 0 | 1 | 2
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        row_ind = math.floor(square / 3)
        row = self.board[row_ind*3:(row_ind+1)*3]
        # print('row', row)
        if all([s == letter for s in row]):
            return True
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        # print('col', column)
        if all([s == letter for s in column]):
            return True
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            # print('diag1', diagonal1)
            if all([s == letter for s in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            # print('diag2', diagonal2)
            if all([s == letter for s in diagonal2]):
                return True
        return False

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == " "]


def play(game, x_player, o_player, print_game=True):

    if print_game:
        game.print_board_nums()

    letter = 'X'
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)
        if game.make_move(square, letter):

            if print_game:
                print(letter + ' makes a move to square {}'.format(square))
                game.print_board()
                print('')

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter  
            letter = 'O' if letter == 'X' else 'X'  

        time.sleep(.8)

    if print_game:
        print('It\'s a tie!')



if __name__ == '__main__':
    x_player = AI('X')
    o_player = HumanPlayer('O')
    t = TicTacToe()
    play(t, x_player, o_player, print_game=True)


| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X makes a move to square 5
|   |   |   |
|   |   | X |
|   |   |   |

O makes a move to square 2
|   |   | O |
|   |   | X |
|   |   |   |

X makes a move to square 0
| X |   | O |
|   |   | X |
|   |   |   |

O makes a move to square 3
| X |   | O |
| O |   | X |
|   |   |   |

X makes a move to square 1
| X | X | O |
| O |   | X |
|   |   |   |

O makes a move to square 4
| X | X | O |
| O | O | X |
|   |   |   |

X makes a move to square 6
| X | X | O |
| O | O | X |
| X |   |   |

O makes a move to square 7
| X | X | O |
| O | O | X |
| X | O |   |

X makes a move to square 8
| X | X | O |
| O | O | X |
| X | O | X |

It's a tie!


In [1]:
def alpha_beta(node, alpha, beta, is_max):
    if type(node) is int:
        return node
    
    if is_max:
        # Max node
        val = float('-inf')
        if 'left' in node:
            val = max(val, alpha_beta(node['left'], alpha, beta, False))
            alpha = max(alpha, val)
        if 'right' in node and beta > alpha:
            val = max(val, alpha_beta(node['right'], alpha, beta, False))
        return val
    else:
        # Min node
        val = float('inf')
        if 'left' in node:
            val = min(val, alpha_beta(node['left'], alpha, beta, True))
            beta = min(beta, val)
        if 'right' in node and beta > alpha:
            val = min(val, alpha_beta(node['right'], alpha, beta, True))
        return val


tree = {
    'value': 'max',
    'left': {
        'value': 'min',
        'left': {
            'value': 'max',
            'left': 2,
            'right': 4
        },
        'right': {
            'value': 'max',
            'left': 6,
            'right': 8
        }
    },
    'right': {
        'value': 'min',
        'left': {
            'value': 'max',
            'left': 1,
            'right': 2
        },
        'right': {
            'value': 'max',
            'left': 10,
            'right': 12
        }
    }
}

result = alpha_beta(tree, float('-inf'), float('inf'), True)
print(result) 


4


In [20]:
!pip install python-constraint

from constraint import Problem

def n_queens(n):
    problem = Problem()
    # Create variables Q1 to Qn
    problem.addVariables(range(1, n+1), range(1, n+1))
    # Add constraints
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i != j:
                problem.addConstraint(lambda x, y, i=i, j=j: x != y and abs(x-y) != abs(i-j), (i, j))
    # Solve the problem
    solution = problem.getSolution()
    if solution:
        return [solution[i] for i in range(1, n+1)]
    else:
        return None
n=8
ans=n_queens(n)
for i in range(1,n+1):
    row=''
    for j in range (1,n+1):
        if j != ans[i-1]:
            row+= (" - ")
        else:
            row+=(" Q ")
    print(row)




 -  -  -  -  -  -  -  Q 
 -  -  -  Q  -  -  -  - 
 Q  -  -  -  -  -  -  - 
 -  -  Q  -  -  -  -  - 
 -  -  -  -  -  Q  -  - 
 -  Q  -  -  -  -  -  - 
 -  -  -  -  -  -  Q  - 
 -  -  -  -  Q  -  -  - 



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import itertools


def get_value(word, substitution):
    s = 0
    factor = 1
    for letter in reversed(word):
        s += factor * substitution[letter]
        factor *= 10
    return s


def solve(equation):
    # split equation in left and right
    left, right = equation.lower().replace(' ', '').split('=')
    # split words in left part
    left = left.split('+')
    # create list of used letters
    letters = set(right)
    for word in left:
        for letter in word:
            letters.add(letter)
    letters = list(letters)

    digits = range(10)
    for perm in itertools.permutations(digits, len(letters)):
        sol = dict(zip(letters, perm))

        if sum(get_value(word, sol) for word in left) == get_value(right, sol):
            print(' + '.join(str(get_value(word, sol)) for word in left) + " = {} (mapping: {})".format(get_value(right, sol), sol))
            
solve('BASE + BALL = GAMES')

2461 + 2455 = 4916 (mapping: {'l': 5, 's': 6, 'm': 9, 'a': 4, 'b': 2, 'e': 1, 'g': 0})
2483 + 2455 = 4938 (mapping: {'l': 5, 's': 8, 'm': 9, 'a': 4, 'b': 2, 'e': 3, 'g': 0})
7483 + 7455 = 14938 (mapping: {'l': 5, 's': 8, 'm': 9, 'a': 4, 'b': 7, 'e': 3, 'g': 1})
